# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [ ]:
!cat publications.tsv

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,citation,paper_url,code_url
0,2012-10-01,Composition-based Classification of Short Meta...,Nucleic Acids Research,"Jiemeng Liu, Haifeng Wang, Hongxing Yang, ...",http://www.ncbi.nlm.nih.gov/pubmed/22941634,http://metacv.sourceforge.net
1,2012-10-01,CRF-based Transcription Factor Binding Site Fi...,BMC Genomics,"Yupeng He, *Yizhe Zhang*, Guangyong Zheng ...",http://www.biomedcentral.com/1471-2164/13/S8/S18/,http://cbb.sjtu.edu.cn/~ccwei/pub/software/CTF...
2,2015-10-01,MOST+: a Motif Finding Approach Combining Geno...,BMC Genomics,"*Yizhe Zhang*, Yupeng He and Chaochun Wei",http://cbb.sjtu.edu.cn/~ccwei/pub/software/MOS...,http://cbb.sjtu.edu.cn/~ccwei/pub/software/MOS...
3,2015-10-01,Learning Dictionary with Spatial and Inter-dic...,"Workshop on representation learning, NIPS","*Yizhe Zhang*, Ricardo Henao, Chunyuan Li,...",./pdf/nips2015.pdf,NONE
4,2016-10-01,Laplacian Hamiltonian Monte Carlo,ECML,"*Yizhe Zhang*, Changyou Chen, Ricardo Hena...",https://dl.acm.org/doi/10.1007/978-3-319-46128...,NONE
5,2016-10-01,Dynamic Poisson Factor Analysis,ICDM,"*Yizhe Zhang*, Changyou Chen, Ricardo Hena...",./pdf/dynpfa_icdm.pdf,./files/dpfa_release.zip
6,2016-10-01,Triply Stochastic Variational Inference for No...,ICDM,"Kai Fan, *Yizhe Zhang*, Lawrence Carin, Ka...",./pdf/VSAE.pdf,./files/dpfa_release.zip
7,2016-10-01,Bayesian Dictionary Learning with Gaussian Pro...,IJCAI,"*Yizhe Zhang*, Ricardo Henao, Chunyuan Li,...",./pdf/VSAE.pdf,./files/gpfa.zip
8,2016-10-01,Learning a Hybrid Architecture for Sequence Re...,AAAI,"*Yizhe Zhang*, Ricardo Henao, Jianling Zho...",./pdf/AAAI_2016_zhang.pdf,./pdf/AAAI_suppl.zip
9,2016-10-01,Generating Text via Adversarial Training.,"Workshop on Adversarial Training, NIPS","*Yizhe Zhang*, Zhe Gan, Lawrence Carin",./pdf/AAAI_2016_zhang.pdf,NONE


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [1]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [4]:
import re
def replace_bold(text):
    """Produce entities within text."""
    return re.sub(r'\*(.*)\*', r'<b>\1</b>', text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    url_slug = "-".join(item.title.strip(".").split())
    md_filename = str(item.pub_date) + "-" + url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + url_slug
    year = item.pub_date[:4]
    item.citation = item.citation.lstrip()
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    # if len(str(item.excerpt)) > 5:
    #     md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(replace_bold(item.venue)) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(replace_bold(item.citation)) + "'"

    if len(str(item.code_url)) > 5:
        md += "\ncode: '" + item.code_url + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    # if len(str(item.excerpt)) > 5:
    #     md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [ ]:
!ls ../_publications/

In [ ]:
!cat ../_publications/2009-10-01-paper-title-number-1.md